# **Ajanın Öğrenme Süreci**
---

![alt text](https://cdn-images-1.medium.com/max/1000/1*aKYFRoEmmKkybqJOvLt2JQ.png)


---




*   Ajan çevreden S(t) durum değerini alır.
*   Bu S(t) değerine göre Ajan bir A(t) eylemi belirler.
*   Bu A(t) eylemi sonucunda bir sonraki durumumuz olan S(t+1) elde edilir.
*   Çevre bu alınan A(t) eyleminin ödülü olan R(t)'yi ajana geri döner.


Bu döngü ajanın eğitim setini deneme yanılma yoluyla oluşturmasını sağlar.




# **Değer Fonksiyonu:**

---

Değer fonksiyonu, ajanın bir durumda alacağı maksimum beklenen toplam getiriyi hesaplayan fonksiyondur.

![değer fonksiyonu](https://cdn-images-1.medium.com/max/1000/0*kvtRAhBZO-h77Iw1.)


---

**örnek değer durumu:**


![örnek değer durumu](https://cdn-images-1.medium.com/max/1000/1*2_JRk-4O523bcOcSy1u31g.png)

# Eylem Değer Fonksiyonu (Action Value Function)

Bir durumda bir hareketin tercih edilmesi durumunda alınması beklenen ödül hesabı.

![alt text](https://cdn-images-1.medium.com/max/1000/1*6IqzImIFK1oEiVWmlu1Esw.png)


# Q-learning:

![alt text](https://cdn-images-1.medium.com/max/1250/1*jmcVWHHbzCxDc-irBy9JTw.png)

Ya durum-eylem düzeyimiz çok büyük ise???


![alt text](https://imgv2-1-f.scribdassets.com/img/document/294823207/original/330c3b7089/1530579908?v=1)

# Deep Learning in Action!

![alt text](https://cdn-images-1.medium.com/max/1000/1*w5GuxedZ9ivRYqM_MLUxOQ.png)



---

**Doom Oyunu**

![alt text](https://cdn-images-1.medium.com/max/800/1*Q4XjhLC0IAOznnk5613PsQ.gif)




**Vizdoom Kütüphanesinin yüklenmesi:**

Linkten işletim sisteminize uygun vizdoom kütüphanesini yükleyebilirsiniz: https://github.com/mwydmuch/ViZDoom/blob/master/doc/Building.md#installationbuilding

**Gerekli Kütüphanelerin Yüklenmesi**

In [ ]:
import tensorflow as tf      
import numpy as np           
import vizdoom  as viz
import os

import random                
import time                  
from skimage import transform

from collections import deque
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

configs_path = "ayarlar"

In [ ]:
### MODEL ÜST DEĞİŞKENLERİ
STATE_SIZE = [84,84,4]      
ACTION_SIZE = 3              
LEARNING_RATE =  0.0002   
DISCOUNT_RATE = 0.95 

### EĞİTİM ÜST DEĞİŞKENLERİ
MAX_EPISODES = 500        
max_steps = 300              
BATCH_SIZE = 64             

# EPSILON GREEDY 
explore_start = 1.0            
explore_stop = 0.01            
decay_rate = 0.0001             

### HAFIZA ÜST DEĞİŞKENLERİ
MEMORY_SIZE = 100000         

episode_render = False

**Çevrenin Oluşturulması**

---

Vizdoom'u kullanarak farklı senaryoları çevre olarak kullanabilir, farklı ayarlar ile çalışılınabilir. Bu yüzden Doom çevresi 2 tane ayar dosyası kullanır:


1.   Çevrenin olası eylemleri ne olabilir, ekran büyüklüğü ne kadar olacak gibi ön tanımlı ayarlarını içeren bir *ayar* dosyası.
2.   Çevrenin hangi senaryoyu uygulayacağını belirttiğimiz bir *senaryo* dosyası.

Biz bu atölye de *basic* [senaryosunu](https://github.com/mwydmuch/ViZDoom/tree/master/scenarios) kullanacağız. Bu senaryoyu ve oyun çevremizin ayarlarını içeren *basic* dosyalarını indirip *ayarlar* klasörünün altına kayıt ediyoruz.

Basic ayarlarından bazılarını aşağıdaki gibi güncelliyoruz:

screen_format = GRAY8

window_visible = false

**Bizim Çevremizin Önemli Özellikleri**

---





*   Canavarlar karşı duvarda random olarak herhangi bir yerde canlanacaklar.
*   Bir bölüm canavar öldüğünde ya da 300 sn geçtiğinde biter.
*   Ajanımız canavarı öldürürse **+101 puan** alır.
*   Her ıskaladığı atış için **-5 puan** alır.
*   Eğer bölüm sonunda canavar ölmemişse **-1 puan** alır.

In [ ]:
class GameEnv():
  
  def __init__(self, screen_format=viz.ScreenFormat.GRAY8, window_visible=False):
    self.game = viz.DoomGame()
    self.load_config(screen_format, window_visible)
    self.game.init()
    
  def load_config(self, screen_format, window_visible,
                  config_file='basic.cfg' , scenario_file = 'basic.wad'):
    scenario_path = os.path.join(configs_path, scenario_file)
    config_path = os.path.join(configs_path, config_file)
    
    self.game.load_config(config_path)
    self.game.set_doom_scenario_path(scenario_path)
    
    self.game.set_screen_format(screen_format)
    self.game.set_window_visible(window_visible)
    
    
  def get_state(self):
    return self.game.get_state().screen_buffer
  
    
  def create_new_episode(self):
    self.game.new_episode()
    
    return self.get_state()
  
  
  def make_action(self, action):
    reward = self.game.make_action(action)
    done = self.game.is_episode_finished()
    
    if done:
      next_state = np.zeros((84,84), dtype=np.int)
    else:
      next_state = self.get_state()
      
    return (reward, done, next_state)

**Bizim Çevremizin 3 olası aksiyonu var:** 

---



*   Sola git
*   Sağa git
*   Ateş et

In [ ]:
LEFT = [1, 0, 0]
RIGHT = [0, 1, 0]
SHOOT = [0, 0, 1]

possible_actions = [LEFT, RIGHT, SHOOT]

**Oyunun her bir frame'inin bir ön işlemden geçirilmesi:**

---

Bizim ajanımızın durum bilgisini oyunun frameleri oluşturmaktadır. Bizim çevremizde her bir frame 240x320 pixelden oluşmaktadır. Her bir adımda bu kadar 240x320 pixel işlemek bizim eğitimimizi yavaşlatıp, bilgisayarın kaynaklarını gereksiz sömürmesine neden olabilir. Bu sebepten framemimizi daha sade bir hale getirmek için bazı ön işlemlerden geçireceğiz.




1.   İlk adımımız resmin renklerini siyah-beyaz yapmak. Çünkü renk bilgisi bizim eğitimimiz için çok önemli bir bilgi değil. (Bu ön adım işlemini çevre ayarlarında screen_format'ı ScreenFormat.GRAY8 yaparak gerçekleştirdik.)
2.   Resmin tavan kısmı yine eğitimimiz için gereksiz bir bilgidir. Bu tavan kısımlarını da kesmek anlamlı bir ön işlem olacaktır.
3.   Son ön işlem adımı ise pixel bilgilerini normalize edip, 84x84 matrise çevirmedir.

In [ ]:
def preprocess_frame(frame):
    # Crop the screen (remove the roof because it contains no information)
    cropped_frame = frame[30:-10,30:-30]
    
    # Normalize Pixel Values
    normalized_frame = cropped_frame/255.0
    
    # Resize
    preprocessed_frame = transform.resize(normalized_frame, [84,84])
    
    return preprocessed_frame

**Frame Yığma İşlemi**

---

Sadece frame üzerinden eğitim yapmanın en büyük problemi hareketin ne yöne doğru gerçekleştiğini göremiyor olmak. Bu problemin önüne geçmek için eğitim modülümü frameleri üst üste yığarak gerçekleştireceğiz. Buradaki amaç 4 farklı frame'i kullanarak eylemin hangi yöne doğru gittiğini belirleyebilmek.

In [ ]:
stack_size = 4 # Eğitim için 4 frame'i üst üste koyuyoruz.

stacked_frames = deque([np.zeros((84,84), dtype=np.int) for i in range(stack_size)], maxlen=4) # queue yapısını kullanıyoruz bu frame yığma işlemi için. Yeni bir frame geldiği zaman en eski frame'i queue'dan çıkarır.

def stack_frames(stacked_frames, state, is_new_episode):
    frame = preprocess_frame(state)
    
    if is_new_episode:
        stacked_frames = deque([np.zeros((84,84), dtype=np.int) for i in range(stack_size)], maxlen=4)
        
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        
        stacked_state = np.stack(stacked_frames, axis=2)
        
    else:
        stacked_frames.append(frame)

        stacked_state = np.stack(stacked_frames, axis=2) 
    
    return stacked_state, stacked_frames

**Derin Öğrenme Modelimiz:**

![alt text](https://cdn-images-1.medium.com/max/1000/1*LglEewHrVsuEGpBun8_KTg.png)

**Derin Öğrenme Eğitiminde kullandığımız Sıralama:**

![alt text](https://cdn-images-1.medium.com/max/1000/1*RFt8MBBkUSPZdolp_WfZFA.png)

In [ ]:
# derin öğrenme modelimiz için yardımcı fonksiyonlar
def create_variable(name, shape, init=None, std=None, wd=None):
    if init is None:
        if std is None:
            std = (2./shape[0])**0.5
            
        init = tf.truncated_normal_initializer(stddev=std)
        
    with tf.device('/cpu:0'):
        new_variables = tf.get_variable(name, shape, initializer=init, dtype=tf.float32)
    
    return new_variables
  
def dense_layer(name, inputs, units, activation = tf.nn.elu, 
                kernel_initializer=tf.contrib.layers.xavier_initializer()):
  
    return tf.layers.dense(inputs, units, 
                           activation = activation, 
                           kernel_initializer= kernel_initializer, 
                           name=name)
  
def conv_layer(name, inputs, filters, kernel_size=[4,4], strides = [2,2],
         kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d()):
  
    return tf.layers.conv2d(inputs, filters, kernel_size, 
                            strides=strides, 
                            kernel_initializer = kernel_initializer, 
                            name=name)
  
def batch_norm_layer(name, conv_layer, training = True, epsilon = 1e-5):
  
    return tf.layers.batch_normalization(conv_layer, 
                                         training = training, 
                                         epsilon= epsilon, 
                                         name=name)

In [ ]:
# derin öğrenme modelimiz
dqn = tf.Graph()

with dqn.as_default():
    with tf.name_scope('input'):
      inputs_ = tf.placeholder(tf.float32, [None, *STATE_SIZE], name="inputs")
      actions_ = tf.placeholder(tf.float32, [None, ACTION_SIZE], name="actions_")
      target_Q = tf.placeholder(tf.float32, [None], name="target")
      
    with tf.variable_scope('conv1') as scope:  # Giriş [84, 84, 4] --> Çıkış [20, 20, 32]
        conv = conv_layer('conv1', inputs_, filters=32, kernel_size = [8,8], strides = [4,4])
        batch_norm = batch_norm_layer('batch_norm1', conv)
        out1 = tf.nn.elu(batch_norm, name="conv1_out")
        
    with tf.variable_scope('conv2') as scope:  # Giriş [20, 20, 32] --> çıkış[9, 9, 64]
        conv = conv_layer('conv2', out1, filters=64)
        batch_norm = batch_norm_layer('batch_norm2', conv)
        out2 = tf.nn.elu(batch_norm, name="conv2_out")
        
    with tf.variable_scope('conv3') as scope:  # Giriş [9, 9, 64] --> çıkış [3, 3, 128]
        conv = conv_layer('conv3', out2, filters=128)
        batch_norm = batch_norm_layer('batch_norm3', conv)
        out3 = tf.nn.elu(batch_norm, name="conv3_out")
        
    with tf.variable_scope('fully_connected1') as scope: # Giriş [3, 3, 128] --> çıkış [1152]
        flatten = tf.layers.flatten(out3) 
        fc1 = dense_layer('fc1', flatten, units = 512)
        
    with tf.variable_scope('output') as scope: # Giriş [1152] --> çıkış [3]
        output_ = dense_layer('output', fc1, units= 3, activation=None)
  
    Q = tf.reduce_sum(tf.multiply(output_, actions_), axis=1)
            
            
    # The loss is the difference between our predicted Q_values and the Q_target
    # Sum(Qtarget - Q)^2
    loss_ = tf.reduce_mean(tf.square(target_Q - Q))
            
    optimizer_ = tf.train.RMSPropOptimizer(LEARNING_RATE).minimize(loss_)

**Deneyim Tekrarlama (Experience Replay)**

---

Bir bölüm boyunca ajanımız farklı farklı durumlar gözlemliyor. Hatırlayacağınız gibi doom'un durum bilgisi anlık ekran(frame) bilgimizdi. Ekran pixelleri bizim durum bilgimiz olunca aynı durumu farklı bölümlerde tekrar denk gelme olasılığımız azalıyor. Fakat biz bu durum bilgisini unutmak istemiyoruz. Bizi başarıya götürecek yolda önemli bir durum olabilir.

Bu sorunun önüne geçmek için ajanın deneyimlerini hafızada saklıyoruz. Eğitim sırasında rastgele olarak bu deneyimlerden seçip ajanı bu rastgele seçilmiş deneyimler ile eğiteceğiz. Bu yönteme ***deneyim tekrarlama (experience replay)*** denir. 

In [ ]:
class Experience():
    def __init__(self, max_size = MEMORY_SIZE):
        self.buffer = deque(maxlen = max_size)
    
    def add(self, experience):
        self.buffer.append(experience)
    
    def sample(self, batch_size = BATCH_SIZE):
        buffer_size = len(self.buffer)
        index = np.random.choice(np.arange(buffer_size), size = batch_size, replace = False)
        
        return [self.buffer[i] for i in index]
      
    def batch_sample(self):
        batch = self.sample()
        
        states = np.array([each[0] for each in batch], ndmin=3)
        actions = np.array([each[1] for each in batch])
        rewards = np.array([each[2] for each in batch]) 
        next_states = np.array([each[3] for each in batch], ndmin=3)
        dones = np.array([each[4] for each in batch])
        
        return (states, actions, rewards, next_states, dones)

In [ ]:
experiences = Experience()
env = GameEnv()

**Epsilon Greedy Stratejisi**

---

Pekiştirmeli öğrenmenin en önemli problemlerinden biri *keşif ve sömürünün* dengede tutulabilmesidir. Eğer çok fazla keşif yaparsa bu sefer çok fazla rasgele eylem deneyecektir ve en iyi öneriyi bulması zorlaşacaktır. Diğer taraftan çok fazla sömürü yaparsa, bu sefer de ajan en iyi eylem olduğunu düşündüğü eylemi sürekli seçer fakat keşif yapmadığı için daha iyi bir eylem olup olmadığını bilemez. Dolayısıyla keşif ve sömürünün belli bir dengede olması gerekir.

Epsilon greedy bu dengeyi gözeten yaklaşımlardan biridir. Belli bir epsilon olasılıkla rasgele bir eylem seçilir. Diğer durumlarda ajan kendisi için en yüksek getiriyi veren eylemi gerçekleştirir.

Biz bu problemde bu epsilon greedy yaklaşımının bir gelişmişini kullandık. Eğitimin başlarında epsilon olasılığı 1 seçilir. Yani ajanımız rastgele aksiyon deneyecektir. Daha sonra zaman geçtikçe bu epsilon olasılığı *decay_rate* oranında düşecektir. Dolayısıyla eğitimin başlarında ajan daha çok *rastgele eylem* deneyerek çok fazla gözlem gerçekleştirir, daha sonra eğitim devam ettikçe bu epsilon oranı düşeceği için *en iyi(greedy) eylemi *seçmeye başlayacaktır.

In [ ]:
def epsilon_greedy_action(explore_start, explore_stop, decay_rate, decay_step, state, actions):
    # Geliştirilmiş epsilon greedy stratejisi kullanılarak epsilon hesabı
    explore_probability = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * decay_step)
    
    if (explore_probability > np.random.rand()): # Random değer eğer epsilon değerimizden küçük ise rastgele hareket seç = keşif(exploration)
        action = random.choice(possible_actions)
    else: # Random değer eğer epsilon değerimizden büyük ise greedy eylemi seç = sömürü(exploitation)

        # Eylemlerin değerlerini hesap et
        Qs = sess.run(output_, feed_dict = {
                          inputs_: state.reshape((1, *state.shape))
                      })
        
        # Maximum değere sahip eylemi seç (greedy eylem)
        choice = np.argmax(Qs)
        action = possible_actions[int(choice)]
                
    return action, explore_probability

Eğitimimizi hafızadaki *batch_size* kadar deneyim ile gerçekleştireceğimizi deneyim tekrarlama başlığında söylemiştik. Öğrenimin başlangıcında eğitimi gerçekleştirebilmek için hafızamısı *batch_size* kadar rastgele eylem ile dolduruyoruz.

In [ ]:
state = env.create_new_episode()
state, stacked_frames = stack_frames(stacked_frames, state, True)

for i in range(BATCH_SIZE):
    action = random.choice(possible_actions)
    reward, done, next_state = env.make_action(action)
    
    if done:
        experiences.add((state, action, reward, next_state, done))
        
        state = env.create_new_episode()
        state, stacked_frames = stack_frames(stacked_frames, state, True)
        
    else:
        next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)
        
        experiences.add((state, action, reward, next_state, done))
        
        state = next_state


**Q-learning**

![alt text](https://cdn-images-1.medium.com/max/1250/1*jmcVWHHbzCxDc-irBy9JTw.png)

**Parametreleştirilmiş TD Hatası**

![alt text](https://cdn-images-1.medium.com/max/1000/1*Zplt-1wTWu_7BGmZCBFjbQ.png)

In [ ]:
def learn_playing():
  target_Qs_batch = []
  states, actions, rewards, next_states, dones = experiences.batch_sample()

  Qs_next_state = sess.run(output_, feed_dict = {inputs_: next_states})

  # Set Q_target = r if the episode ends at s+1, otherwise set Q_target = r + DISCOUNT_RATE*maxQ(s', a')
  for i in range(0, BATCH_SIZE):
      terminal = dones[i]

      if terminal: # Eğer terminal durumunda ise Q_target = r
          target_Qs_batch.append(rewards[i]) 
          
      else: # Q_target = r + DISCOUNT_RATE*maxQ(s', a')
          target = rewards[i] + DISCOUNT_RATE * np.max(Qs_next_state[i])
          target_Qs_batch.append(target)

  targets = np.array([each for each in target_Qs_batch])

  loss, _ = sess.run([loss_, optimizer_],
                      feed_dict={inputs_: states,
                                 target_Q: targets,
                                 actions_: actions})
  
  return loss

In [ ]:
def run_an_episode(episode, env, decay_step, stacked_frames):
  total_reward = 0
  step = 0
  episode_rewards = []
  
  state = env.create_new_episode()
  state, stacked_frames = stack_frames(stacked_frames, state, True)
  done = False
  
  while(not done):
    step += 1
    decay_step +=1

    action, explore_probability = epsilon_greedy_action(explore_start, explore_stop, decay_rate, decay_step, state, possible_actions)

    reward, done, next_state = env.make_action(action)

    episode_rewards.append(reward)

    if done:
        next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)

        total_reward = np.sum(episode_rewards)

        print('Episode: {}'.format(episode),
                  'Total reward: {}'.format(total_reward),
                  'Training loss: {:.4f}'.format(loss),
                  'Explore P: {:.4f}'.format(explore_probability))

        experiences.add((state, action, reward, next_state, done))
    else:
        if step % 10 == 0:
          print('Episode: {}'.format(episode),
                  'Step: {}'.format(step),
                  'Action: {}'.format(action),
                  'Training loss: {:.4f}'.format(loss),
                  'Explore P: {:.4f}'.format(explore_probability))

        next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)

        experiences.add((state, action, reward, next_state, done))

        state = next_state

    loss = learn_playing()
  
  return total_reward, decay_step, stacked_frames

In [ ]:
with dqn.as_default():
  saver = tf.train.Saver()
  sess = tf.Session()

  try:
      checkpoint_dir = tf.train.latest_checkpoint(checkpoint_dir="./models/")
      saver.restore(sess, save_path=checkpoint_dir)
      print("Restored checkpoint from:", checkpoint_dir)
  except:
      print("Initializing variables")
      sess.run(tf.global_variables_initializer())

In [ ]:
decay_step = 0

for episode in range(MAX_EPISODES):
    total_reward, decay_step, stacked_frames = run_an_episode(episode, env, decay_step, stacked_frames)
    
    # Save model every 5 episodes
    if episode % 5 == 0:
        save_path = saver.save(sess, "./models/model.ckpt")
        print("\n Episode: %d, Model Saved" % (episode))

    print('*'*70)
    print('Episode Finished! Total Reward is %d\n' % (total_reward))
    print('*'*70) 